In [1]:
import pandas as pd
import numpy as np

import requests

import json

from datetime import datetime, timedelta

from pprint import  pprint

import plotly.express as px


In [2]:
lang = input('¿En español (es) o en ingles (en)?')

restaDia = float(input('¿Cuantos días atrás?:____________'))
input_año = int(input('¿Qué año?:_____________'))

In [3]:
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Host": "apidatos.ree.es"
}

In [4]:
def generacion_datos(lang):

    now = datetime.now()
    ultima_fecha = (now - timedelta(days = restaDia)).replace(year = input_año).strftime('%Y-%m-%d')
    
    hoy = now.replace(year = input_año).strftime('%Y-%m-%d')
    
    query = f'start_date={ultima_fecha}T00:00&end_date={hoy}T23:59&time_trunc=day'


    endpoint = f"https://apidatos.ree.es/{lang}/datos/generacion/estructura-generacion?{query}"
    response = requests.get(url= endpoint, headers= headers)
    data = response.json()

    lista_nombres = list()
    lista_tipos = list()
    lista_valores = list()
    lista_porcentajes = list()
    lista_fechas = list()
    



    for dato in data['included']:
        nombre = dato['attributes']['title']
        tipo = dato['attributes']['type']


        for i in dato['attributes']['values']:
            valor = i['value']

            porcentaje = i['percentage']

            fecha = i['datetime']
            fecha = pd.to_datetime(fecha)
            

            fecha =  fecha.strftime("%d/%m/%Y")

            lista_nombres.append(nombre)
            lista_tipos.append(tipo)
            lista_valores.append(valor)
            lista_porcentajes.append(porcentaje)
            lista_fechas.append(fecha)
            

    df_generacion = pd.DataFrame()
    df_generacion['nombre']               = lista_nombres
    df_generacion['tipo de energía']      = lista_tipos
    df_generacion['Valores']              = lista_valores
    df_generacion["Porcentaje"]           = lista_porcentajes
    df_generacion["Fecha actualización"]  = lista_fechas
   


    return df_generacion
    
df_generacion = generacion_datos(lang)

df_generacion

,nombre,tipo de energía,Valores,Porcentaje,Fecha actualización
0,Hidráulica,Renovable,24805.877,0.033842,14/09/2023
1,Hidráulica,Renovable,31052.893,0.044154,15/09/2023
2,Hidráulica,Renovable,24153.141,0.039668,16/09/2023
3,Hidráulica,Renovable,15558.041,0.023926,17/09/2023
4,Hidráulica,Renovable,31779.909,0.047813,18/09/2023
...,...,...,...,...,...
523,Generación total,Generación total,688814.909,1.000000,10/10/2023
524,Generación total,Generación total,656140.296,1.000000,11/10/2023
525,Generación total,Generación total,635614.935,1.000000,12/10/2023
526,Generación total,Generación total,619507.095,1.000000,13/10/2023


In [5]:
df_generacion.to_csv('generacion_estructura.csv')

In [6]:
px.line(data_frame = df_generacion, x = 'Fecha actualización', y = 'Valores', color = 'nombre',
        line_group= 'tipo de energía',
        title= 'Tipo de energía',
        markers = True)

In [18]:
px.box(data_frame = df_generacion,
       x = 'Valores',
       y = 'nombre',
       color= 'nombre')

In [9]:
px.box(data_frame= df_generacion,
       x = 'Valores',
       y = 'tipo de energía',
       title= 'Box tipo de energía',
       color = 'tipo de energía')

In [32]:
px.histogram(data_frame= df_generacion,
             x = 'Valores',
             y = 'Porcentaje',
             color = 'nombre',
             nbins= 200)

In [10]:
px.histogram(data_frame= df_generacion,
             x = 'Valores',
             y = 'Porcentaje',
             color = 'tipo de energía',
             title= 'Histograma por su tipo de energía',
             facet_col= 'tipo de energía',
             nbins= 50)

In [37]:
px.violin(data_frame=df_generacion,
             x = 'nombre',
             y = 'Valores',
             color = 'nombre',
             points= 'suspectedoutliers'
             )

In [39]:
px.scatter(data_frame= df_generacion,
           x = 'Valores',
           y = 'Porcentaje',
           color = 'nombre',
           hover_name= 'nombre' )